Passive Model

In [ ]:
import cv2
import os
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

#Date Preprocessing

FRAME_COUNT = 16
FRAME_SIZE = (128, 128)
DATASET_PATH = '/content/drive/MyDrive/files'

data = []
labels = []

allowed_extensions = ('.mp4', '.avi', '.mov', '.mkv')

def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < FRAME_COUNT and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, FRAME_SIZE)
        frames.append(frame)
    cap.release()

    while len(frames) < FRAME_COUNT:
        frames.append(frames[-1])

    frames = np.array(frames, dtype="float32") / 255.0
    return frames

for category in ['mask', 'mask_cut', 'outline', 'real']:
    video_dir = os.path.join(DATASET_PATH, category)
    label = 0 if category == 'real' else 1

    if not os.path.exists(video_dir):
        print(f"Directory does not exist: {video_dir}")
        continue

    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)

        if video_file.endswith(allowed_extensions):
            print(f"Processing {video_path}...")
            frames = preprocess_video(video_path)
            data.append(frames)
            labels.append(label)
        else:
            print(f"Skipping non-video file: {video_path}")

data = np.array(data)
labels = np.array(labels)

print(f"Total number of samples: {len(data)}")
print(f"Number of real samples: {np.sum(labels == 0)}")
print(f"Number of spoof samples: {np.sum(labels == 1)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing /content/drive/MyDrive/files/mask/5.mp4...
Processing /content/drive/MyDrive/files/mask/2.mp4...
Processing /content/drive/MyDrive/files/mask/6.mp4...
Processing /content/drive/MyDrive/files/mask/4.mp4...
Processing /content/drive/MyDrive/files/mask/3.mp4...
Processing /content/drive/MyDrive/files/mask/0.mp4...
Processing /content/drive/MyDrive/files/mask/7.mp4...
Processing /content/drive/MyDrive/files/mask/1.mp4...
Processing /content/drive/MyDrive/files/mask/8.mp4...
Processing /content/drive/MyDrive/files/mask/9.mp4...
Processing /content/drive/MyDrive/files/mask_cut/2.mp4...
Processing /content/drive/MyDrive/files/mask_cut/4.mp4...
Processing /content/drive/MyDrive/files/mask_cut/1.mp4...
Processing /content/drive/MyDrive/files/mask_cut/5.mp4...
Processing /content/drive/MyDrive/files/mask_cut/3.mp4...
Processing /content/drive/MyDrive/files/m

In [ ]:
from keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from keras.models import Model

input_shape = (FRAME_COUNT, FRAME_SIZE[0], FRAME_SIZE[1], 3)

# Input layer
input_layer = Input(shape=input_shape)

# 3D Convolutional layers
x = Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)

x = Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

model_3dcnn = Model(inputs=input_layer, outputs=output_layer)
model_3dcnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_3dcnn.summary()

model_3dcnn.save('passive.h5')


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 16, 128, 128, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 16, 128, 128, 32)    │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_2 (MaxPooling3D)       │ (None, 8, 64, 64, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_3 (Conv3D)                    │ (None, 8, 64, 64, 32)       │          27,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_3 (MaxPooling3D)       │ (None, 4, 32, 32, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 131072)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      67,109,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 67,140,193 (256.12 MB)

 Trainable params: 67,140,193 (256.12 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

model_3dcnn.fit(X_train, y_train, epochs=5, batch_size=8, validation_data=(X_val, y_val))


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 509ms/step - accuracy: 0.6500 - loss: 11.2263 - val_accuracy: 0.8750 - val_loss: 1.2691
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.6958 - loss: 1.6566 - val_accuracy: 0.8750 - val_loss: 0.4267
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.9083 - loss: 0.2921 - val_accuracy: 0.8750 - val_loss: 0.1685
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 1.0000 - loss: 0.0969 - val_accuracy: 0.8750 - val_loss: 0.3567
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.9417 - loss: 0.0976 - val_accuracy: 1.0000 - val_loss: 0.0073


Active Model

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential

# Define the LSTM model
model_lstm = Sequential()

# Input shape is (number of frames, height * width), e.g., 16 frames of flattened 128x128 grayscale frames
model_lstm.add(LSTM(64, input_shape=(FRAME_COUNT, FRAME_SIZE[0] * FRAME_SIZE[1]), return_sequences=True))
model_lstm.add(Dropout(0.5))

model_lstm.add(LSTM(32))
model_lstm.add(Dropout(0.5))

# Fully connected layer
model_lstm.add(Dense(64, activation='relu'))

# Output layer for binary classification
model_lstm.add(Dense(1, activation='sigmoid'))

# Compile the model
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model_lstm.summary()
model_lstm.save('active.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 16, 64)              │       4,210,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 16, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,225,537 (16.12 MB)

 Trainable params: 4,225,537 (16.12 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the LSTM model
model_lstm.fit(X_train, y_train, epochs=50, batch_size=8,validation_data=(X_val, y_val))
